In [ ]:
from Crypto.Cipher import AES
import os
from Crypto.Util.Padding import pad, unpad
from datetime import datetime, timedelta


KEY = "blueblue"*2
FLAG = "crypto{flag_is_here}"


# @chal.route('/flipping_cookie/check_admin/<cookie>/<iv>/')
def check_admin(cookie, iv):
    cookie = bytes.fromhex(cookie)
    iv = bytes.fromhex(iv)

    try:
        cipher = AES.new(KEY, AES.MODE_CBC, iv)
        decrypted = cipher.decrypt(cookie)
        unpadded = unpad(decrypted, 16)
    except ValueError as e:
        return {"error": str(e)}

    if b"admin=True" in unpadded.split(b";"):
        return {"flag": FLAG}
    else:
        return {"error": "Only admin can read the flag"}


# @chal.route('/flipping_cookie/get_cookie/')
def get_cookie():
    expires_at = (datetime.today() + timedelta(days=1)).strftime("%s")
    cookie = f"admin=False;expiry={expires_at}".encode()

    iv = os.urandom(16)
    padded = pad(cookie, 16)
    cipher = AES.new(KEY, AES.MODE_CBC, iv)
    encrypted = cipher.encrypt(padded)
    ciphertext = iv.hex() + encrypted.hex()

    return {"cookie": ciphertext}

In [8]:
from Crypto.Cipher import AES
import os
from Crypto.Util.Padding import pad, unpad
from datetime import datetime, timedelta
import requests

ENCODE_URL = "https://aes.cryptohack.org/flipping_cookie/get_cookie/"
DECODE_URL = "https://aes.cryptohack.org/flipping_cookie/check_admin/"

def get_respone(url):
    respone = requests.get(url).json()
    keys = respone.keys()
    for k in ("ciphertext", "plaintext", "cookie", "flag"):
        if k in keys:
            return respone[k]
        
def hex_XOR(a, b):
    assert len(a) == len(b)
    return bytes(x ^ y for x, y in zip(bytes.fromhex(a), bytes.fromhex(b))).hex()

# Get cookie
cookie = get_respone(ENCODE_URL)
print(cookie)

assert len(cookie) % 32 == 0

iv = cookie[:32]
cookie_blocks = [cookie[i:i+32] for i in range(32, len(cookie), 32)]

# text = admin=True;, 
text_false = b"admin=False".hex() + (f"{chr(0)}" * 5).encode('utf-8').hex()
text_true  = b"admin=True;".hex() + (f"{chr(0)}" * 5).encode('utf-8').hex()

# IV + newIV = True + False
# ==> newIV = True + False + IV

newIV = hex_XOR(hex_XOR(text_true, text_false), iv)
print(newIV)

# Get Flag
FLAG = get_respone(DECODE_URL + cookie[32:] + "/" + newIV + "/")
print(FLAG)


300d2c51c6999148ee1df7b07ad56c5dd82272a6a5beff11007d95eec978b668a410af9be54b91d37cf02138b53ee15c
300d2c51c699835bf70ba9b07ad56c5d
crypto{4u7h3n71c4710n_15_3553n714l}
